In [8]:
import csv, time, lxml, json, requests
import pandas as pd
from pandas.io.json import json_normalize
import bs4 as bs
import urllib3 as urllib
import matplotlib.pyplot as plt
%matplotlib inline

In [18]:
def get_urls():
    Base_url = "http://www.scstrade.com"
 
    # Build a dictionary of companies and their abbreviated names 
    companies = {'Habib Bank Limited':'HBL', 'Engro Chemical':'ENGRO'}

    # Create a list of the news section urls of the respective companies 
    return ['http://www.scstrade.com/stockscreening/SS_CompanySnapShot.aspx?symbol={}'.format(v) 
                for k,v in companies.items()]

In [21]:
def jsontodataframe(): #collect OHLC data from scstrade

    companies = {'Habib Bank Limited':'HBL'}
    url = 'http://www.scstrade.com/stockscreening/SS_CompanySnapShotHP.aspx/chart'

    payload = {"par":"ENGRO","date1":"01/01/2019","date2":"06/01/2019","rows":20,"page":1,"sidx":"trading_Date",
    "sord":"desc"}

    json_data = requests.post(url, json=payload).json() #download the json POST request from scstrade
    json_normalize(json_data)
    df = pd.DataFrame(json_data) #convert the json to pandas dataframe

    df = pd.io.json.json_normalize(json_data['d'], errors='ignore')

    df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Change'] #rename the columns to better names

    #     df['ID'] = companies.values() 
    #     df.head() 
    
    df['Date'] = df['Date'].str.strip('/Date()')
    df['Date'] = pd.to_datetime(df['Date'], origin='unix', unit='ms') #convert unix timestamp to pandas datetime and set the index
    df.set_index(['Date'], inplace=True)
    df.to_csv("/home/duke/PSMSA/SCSTrade/OHLC_values.csv") #save .csv file for later usage
    return df.head()


In [22]:
jsontodataframe()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2019-05-29 19:00:00,289.75,294.01,281.55,283.80,1358200,-3.88
2019-05-28 19:00:00,280.99,290.50,280.99,287.68,1321300,11.01
2019-05-27 19:00:00,281.50,281.50,274.80,276.67,1062000,-5.04
2019-05-26 19:00:00,280.00,284.90,279.61,281.71,794400,1.99
2019-05-23 19:00:00,280.12,284.75,277.05,279.72,609300,-0.40
